In [32]:
from tensorflow import keras
from tf_keras_vis.activation_maximization import ActivationMaximization
from tensorflow import random

In [33]:
model =  keras.models.load_model("model-files")

In [34]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 244, 244, 64)      9472      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 122, 122, 64)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 116, 116, 32)      100384    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 58, 58, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 52, 52, 32)        50208     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 20, 20, 32)       

In [35]:
from tf_keras_vis.utils.model_modifiers import ExtractIntermediateLayer, ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore
from tf_keras_vis.activation_maximization import ActivationMaximization
from tf_keras_vis.activation_maximization.callbacks import Progress

max_filters = 20

layer_names = ["conv2d_10", "conv2d_11", "conv2d_12", "conv2d_13", "conv2d_14"]
activations = {}
for layer_name in layer_names:
    n_filters = min(model.get_layer(layer_name).filters, max_filters)

    extract_intermediate_layer = ExtractIntermediateLayer(index_or_name=layer_name)
    replace2linear = ReplaceToLinear()
    activation_maximization = ActivationMaximization(model,model_modifier=[extract_intermediate_layer, replace2linear],clone=True)
    
    seed_input_shape  = (n_filters, model.input_shape[1], model.input_shape[2],model.input_shape[3])
    seed_input = random.uniform(seed_input_shape, 0, 255)
    scores = CategoricalScore(list(range(n_filters)))
    activations[layer_name] = activation_maximization(scores,steps = 400 ,seed_input = seed_input, callbacks=[Progress()])


 14/400 [>.............................] - ETA: 19:34 - Score: -58.3710 - TotalVariation2D: 19.7432 - Norm: 22.9514

KeyboardInterrupt: 

In [ ]:
from numpy import savez_compressed
for layer_name in layer_names:
    savez_compressed("activation-files/activations_" + layer_name +'.npz', activations[layer_name])